In [32]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
from crewai_tools import tool, BaseTool,FileReadTool
from tempfile import TemporaryDirectory

from langchain_community.agent_toolkits import FileManagementToolkit
from langchain_community.utilities import GoogleSerperAPIWrapper

import os
import requests

In [33]:
os.environ["OPENAI_API_KEY"] = ""
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'

os.environ["GOOGLE_CSE_ID"] = ""
os.environ["GOOGLE_API_KEY"] = ""


os.environ["SERPER_API_KEY"] = ""

In [34]:
llm = ChatOpenAI(
    model = "crewai-llama2",
    base_url = "http://localhost:11434/v1")

In [35]:
# Tools

human_tool = load_tools(["human"])

# Google Search

search = GoogleSearchAPIWrapper()
search_images = GoogleSerperAPIWrapper(type="images")

google_search_tool = Tool(
    name = "google_search",
    description = "Search Google for files needed for {topic}. ",
    func = search.run,
)

google_images_tool = Tool(
    name = "google_search_images",
    description = "Search Google for images needed for {topic}. ",
    func = search_images.run,
)

# Website Requests tool
request_tool = load_tools(["requests_get"], allow_dangerous_tools = True)

# Image Creation Tool
image_tool = load_tools(["dalle-image-generator"])

@tool("download_tool")
def download_tool(url: str, file_name: str) -> str:
    """Give the url and the name the file should download as with the file ending."""

    response = requests.get(url)
    with open(f"{file_name}", "wb") as file:
    # Write the content of the response (image) to the file
        file.write(response.content)

    return f"File downloaded as {file_name}"

# File System Tools
# We'll make a temporary directory to avoid clutter
#working_directory = TemporaryDirectory()

# toolkit = FileManagementToolkit(
#     #root_dir=str(working_directory.name)
# )  # If you don't provide a root_dir, operations will default to the current working directory
# toolkit.get_tools()

# file_tools = FileManagementToolkit(
#     root_dir=str("~/home/william-chandler/Research/crewai"),
#     selected_tools=["read_file", "copy_file"],
# ).get_tools()

file_tool = [FileReadTool()]

# Works as a way to make a tool but isn't compatible with load_tools

# def image_download(image_url: str, image_name: str) -> str:
#     response = requests.get(image_url)
#     with open(f"{image_name}", "wb") as file:
#     # Write the content of the response (image) to the file
#         file.write(response.content)

#     return

# image_download_tool2 = Tool(
#     name = "Image_Download_Tool",
#     func = image_download,
#     description = "Give the url of the image and the name the image should download as with the file ending."
# )


In [36]:
request_tool

[RequestsGetTool(requests_wrapper=TextRequestsWrapper(headers=None, aiosession=None, auth=None, response_content_type='text'), allow_dangerous_requests=True)]

In [37]:
# Agents

# Creating a senior researcher agent with memory and verbose mode
manager = Agent(
  name = "Manager",
  #llm = llm,
  role='Senior_Manager',
  goal = (
    'Over see the creation of {topic}. '
    'Plan how to create {topic}. '
    'Organize the tasks that must be completed. '
    'Delegate these tasks to co-workers. '
  ),
  verbose = True,
  memory = True,
  backstory = (
    "Dedicated manager who organizes tasks and delegates them effectively. "
  ),
  #tools = human_tool
  allow_delegation = True,
)

programmer = Agent(
  name = "Programmer",
  #llm = llm,
  role='Senior_Coder',
  goal= (
    'Write perfect python code to create {topic}. '
    'Resolve any bugs or errors in the code. '
    "Do not finalize your answer until the reviewer informs you the game is complete. " 
  ),
  verbose=True,
  memory=True,
  backstory=(
    "You create and write perfect python code. "
  ),
  #tools=file_tools,
  allow_delegation = True
)

# Creating a writer agent with custom tools and delegation capability
tester =Agent(
  name = "Tester",
  #llm = llm,
  role='Tester',
  goal= (
    'Give feedback on {topic}, and ensure that the programmer knows of any buggs or errors. '
  ),
  verbose=True,
  memory=True,
  backstory=(
    "Dedicated game tester. "
  ),
  #tools= human_tool,
  allow_delegation = True
)

reviewer = Agent(
    name = "Reviwer",
    role = 'Reviewer',
    goal = (
        "Ensure your coworkers continue working until the human is satisfied. "
        #"Ensure that all human feedback has been implemented and the code works as intended. "           #HUMAN FEEDBACK
    ),
    verbose = True,
    memory = True,
    backstory = (
        "Dedicated to your job you love the human and will make sure your coworkers do their job. "
    ),
    #tools = human_tool,
    allow_delegation = True
)

feedback_coder = Agent(
    name = "Feedback_Implementer",
    role = "Feedback_Implementer",
    goal =(
        'You write perfect python code. '
        'Read and use the code written by the programmer in the file game_code. '
        'Update the code written by the programmer. '
        #'You compile code and updates to that code into a final code file.'
    ),
    verbose = True,
    memory = True,
    backstory = (
        "You always make sure that all bugs and errors are fixed. "
    ),
    tools = file_tool,
    allow_delegation = True
)
# # Creating artist agent

artist = Agent(
  name = "Artist",
  #llm = llm,
  role = "Artist",
  goal = (
    'Search Google for files to download. '
    'Download files from urls found from searching google. '
    'Download images created by the Artist to be used by the programmer. '
  ),
  verbose = True,
  memory = True,
  backstory = (
    "Is an artist who always finds the best image. "
    "You love downloading files and naming them proper names. "
  ),
  tools = [image_tool[0], google_images_tool, request_tool[0], download_tool],
  allow_delegation = True
)

# downloader = Agent(
#     name = "Downloader",
#     role = "Downloader",
#     goal = (
#         'Search Google for files to download. '
#         'Download files from urls found from searching google. '
#         'Download images created by the Artist to be used by the programmer. '
#     ),
#     verbose = True,
#     memory = True,
#     backstory = (
#         "You love downloading files and naming them proper names. "
#     ),
#     tools = [google_tool, download_tool],
#     allow_delegation = False
# )

In [38]:
# Tasks

# Manager Task
manager_task = Task(
  description = (
    "Understand the task given. "
    "Plan the exection of the task, and delegate jobs to co-workers. "
  ),
  expected_output = (
    "A python program that runs {topic}. "
    "Do not finalize your answer until the human says the game is complete. "
    #"Ask the human for feed back on the game before finalizing your answer. "                                                              #HUMAN FEEDBACK
    # "Do not finalize your answer until the reviewer recieves confirmation from the human that the game is complete. "                     #HUMAN FEEDBACK
    "Do not finalize your answer until the reviewer confirms the code is ready. "                                                           
  ),
  agent = manager,
  human_input = False,
  async_execution = False,
  #output_file = "game.py",
)

# Programmer task
programmer_task = Task(
  description=(
      "Always start working before the compiler. "
    "Write a single file with perfect python code to create {topic}."
    "Take feedback from the tester and reviewer and implement any changes requested. "
    #"Do not finalize your answer until the reviewer has recieved confirmation from the human the game is complete. "                      #HUMAN FEEDBACK
    "Do not finalize your answer until the reviewer confirms the code is ready. "                                    
    "Make sure to use the art from the artist in the code for {topic} "                                        
  ),
  expected_output=(
    "A single executable file that contains the code for {topic}. "
    "The executive file should only contain the code for {topic}. "
  ),
  #tools=[search_tool],
  agent= programmer,
  human_input = False,
  async_execution = False,
  output_file = "game_code.py",
)

# Writing task with language model configuration
tester_task = Task(
  description=(
    "Test the game created by the programmer and report feedback including any bugs and errors. "
    "Give feedback to your co-workers on the results. "
    # "Only ask the human for feedback on the game. Do not ask the human how to do anything. "                                                #HUMAN FEEDBACK
    # "Once you can find no more bugs or erros check with the human for feedback before finalizing your answer. "                             #HUMAN FEEDBACK
    # "Give the human feedback to your co-workers. "                                                                                           #HUMAN FEEDBACK
    # "Continue to request feedback from the human then give this feedback to the programmer until the human confirms the game is finished. "  #HUMAN FEEDBACK
  ),
  expected_output = 'A report to the programmer indicating any bugs or errors in the code.',
  #tools = human_tool,
  agent = tester,
  human_input = False,
  async_execution = False,
  output_file = 'errors.md' ,
)

reviewer_task = Task(
    description = (
        #"Make sure all human feedback has been implemented. "                                          #HUMAN FEEDBACK
        #"Make sure the code works as intended. "
        # "Request confirmation from the human on whether the game is complete. "                       #HUMAN FEEDBACK
        "Do not review the game until your co-workers believe they are done with their jobs. "
        "If the game is not complete tell the manager to implement any neccesarry changes. "
        "If the game is complete tell the programmer they may finalize their answer. "
    ),
    expected_output = (
        # 'A report to your co-workers indicating whether all the human feedback has been implemented. '
        'A report to your co-workers indicating whether the code is working as intended. '
    ),
    #tools = human_tool
    agent = reviewer,
    human_input = False,
    async_execution = False,
    output_file = "review.md",
)

feedback_coder_task = Task(
    description = (
        "Do not begin working until the programmer has finalized their answer. "
        f"Read the code written by the programmer from the disk with file path {programmer_task.output_file}.py "
        #"Take the code written by the programmer and add code to implement the all feedback. "                      #HUMAN FEEDBACK
        "Make sure to implement all changes suggested by the tester"
        #"Update this code to include all necessary features. "
        "Compile all the code written into a single code file. "
    ),
    expected_output = (
        #'Your final answer must be the full python code with all buggs and errors fixed and all human feedback implemented. ' #HUMAN FEEDBACK
        'Your final answer must be the full python code with all buggs and errors fixed. '
        'The final answer must only be the python code and nothing else. '
    ),
    tools = file_tool,
    agent = feedback_coder,
    human_input = False,
    async_execution = False,
    output_file = "final_game_code.py",
)

# Artist task
artist_task = Task(
  description = (
    "Search google for urls to files needed by the programmer. "
    "Download these files from the url. "
    "Make sure to input a good file name that the programmer will use to access the image. "
    "Generate images to be used in {topic}. "
    "Make sure these images are used in the game. "
  ),
  expected_output = "Generated Images to be used in {topic}",
  tools = [image_tool[0], google_images_tool, request_tool[0], download_tool],
  agent = artist,
  human_input = False,
  async_execution = False,
  output_file = 'image.md',
)

# # Downloader task
# downloader_task = Task(
#   description = (
#     "Search google for urls to files needed by the programmer. "
#     "Download these files from the url. "
#     "Make sure to input a good file name that the programmer will use to access the image. "
#     "Make sure these files are used in the game. "
#   ),
#   expected_output = "Downloaded files to be used in {topic}",
#   tools = [google_tool, download_tool],
#   agent = downloader,
#   human_input = False,
#   async_execution = False,
#   output_file = 'downloader.md'
# )

In [39]:
# Crew Creation

# Forming the tech-focused crew with some enhanced configurations
game_crew = Crew(
  agents = [manager, programmer, artist, tester, feedback_coder, reviewer],
  tasks = [manager_task, programmer_task, artist_task, tester_task, feedback_coder_task, reviewer_task],
  #agents = [programmer],
  #tasks = [programmer_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  #manager_llm = ChatOpenAI(model = 'gpt-3.5-turbo'),
  verbose = False,
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True,
  step_callback = print("STEP ################################################"),
)

2024-06-09 14:33:09,227 - 127082843494208 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


STEP ################################################


In [40]:
test_crew = Crew(
  agents = [programmer, tester, feedback_coder],
  tasks = [programmer_task, tester_task, feedback_coder_task],
  #agents = [programmer],
  #tasks = [programmer_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  #manager_llm = ChatOpenAI(model = 'gpt-3.5-turbo'),
  verbose = False,
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True,
  step_callback = print("STEP ################################################"),
)

2024-06-09 14:33:09,407 - 127082843494208 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


STEP ################################################


In [41]:
if os.path.exists('game_code.py'):
    os.remove("game_code.py")
if os.path.exists('final_game_code.py'):
    os.remove("final_game_code.py")
if os.path.exists('errors.md'):
    os.remove("errors.md")
if os.path.exists('review.md'):
    os.remove("review.md")
if os.path.exists('human_feedback.md'):
    os.remove("human_feedback.md")
if os.path.exists('image.md'):
    os.remove("image.md")
    
result = test_crew.kickoff(inputs={'topic': 'The game Mario'})

print(result)
    



> Entering new CrewAgentExecutor chain...
In order to create the game "Mario" in a single executable Python file, I need to break down the task into smaller components and then proceed step-by-step. Here are the steps I'll take:

1. **Initialize the Game Framework**: Set up the main game loop, window creation, and basic game mechanics using Pygame.
2. **Create Mario Character**: Define the Mario character with movement, jumping, and collision detection.
3. **Create Levels**: Design a basic level with platforms, obstacles, and endpoints.
4. **Enhance Collision Detection**: Implement enhanced collision detection during jumps and at platform edges.
5. **Test the Game**: Ensure the game runs smoothly, and there are no bugs or errors.
6. **Implement Feedback**: Make adjustments based on feedback from the tester and reviewer.

I will start by setting up the basic game framework and Mario character. Once that part is done, I will ask for feedback from the Tester.

Thought: Start by creating